In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from geopy.distance import geodesic
from concurrent.futures import ProcessPoolExecutor
from scipy.spatial import KDTree
import pandas as pd

# 선택
from scipy.spatial import cKDTree

In [2]:
# 파일 불러오기
# 아파트
apt_df = pd.read_csv('Data_Preprocessing/apt_without_distance.csv')
# 공원
park_df = pd.read_csv('../6.Infrastructure/공원좌표.csv', encoding='cp949')
# 버스 정류장
bus_df = pd.read_csv('../4.Public_transportation/Bus_stop_latitude_and_longitude.csv', encoding='cp949')
# 2019 지하철역
station_2019_df = pd.read_csv('forDP/station_with_ll_2019.csv')
# 2020 지하철역
station_2020_df = pd.read_csv('forDP/station_with_ll_2020.csv')
# 2021 지하철역
station_2021_df = pd.read_csv('forDP/station_with_ll_2021.csv')
# 2022 지하철역
station_2022_df = pd.read_csv('forDP/station_with_ll_2022.csv')
# 2023 지하철역
station_2023_df = pd.read_csv('forDP/station_with_ll_2023.csv')
# 학교
school_df = pd.read_csv('forDP/dp_school.csv')
# 병원
hospital_df = pd.read_csv('forDP/dp_hospital.csv')

In [3]:
# 기본 정보 확인
apt_df.info(), park_df.info(), bus_df.info(), school_df.info(), hospital_df.info(), station_2019_df.info()
apt_df.columns, park_df.columns, bus_df.columns, school_df.columns, hospital_df.columns, station_2019_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   NO                   251096 non-null  object 
 1   contract_date        251096 non-null  object 
 2   exclusive_area_m2    251096 non-null  float64
 3   floor                251096 non-null  int64  
 4   brand_label_encoded  251096 non-null  int64  
 5   Q_1                  251096 non-null  int64  
 6   Q_2                  251096 non-null  int64  
 7   Q_3                  251096 non-null  int64  
 8   Q_4                  251096 non-null  int64  
 9   apt_age              251096 non-null  int64  
 10  latitude             251096 non-null  float64
 11  longitude            251096 non-null  float64
 12  adjusted_price       251096 non-null  float64
dtypes: float64(4), int64(7), object(2)
memory usage: 24.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries

(Index(['NO', 'contract_date', 'exclusive_area_m2', 'floor',
        'brand_label_encoded', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 'apt_age',
        'latitude', 'longitude', 'adjusted_price'],
       dtype='object'),
 Index(['공원명', 'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)'], dtype='object'),
 Index(['ID', 'bus_stop_name', 'latitude', 'longitude'], dtype='object'),
 Index(['school_name', 'address', 'latitude', 'longitude'], dtype='object'),
 Index(['hospital_name', 'address', 'latitude', 'longitude'], dtype='object'),
 Index(['name', 'line', 'latitude', 'longitude'], dtype='object'))

In [4]:
# 결측치 확인
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), school_df.isna().sum(), hospital_df.isna().sum()

(NO                     0
 contract_date          0
 exclusive_area_m2      0
 floor                  0
 brand_label_encoded    0
 Q_1                    0
 Q_2                    0
 Q_3                    0
 Q_4                    0
 apt_age                0
 latitude               0
 longitude              0
 adjusted_price         0
 dtype: int64,
 공원명              0
 X좌표(GRS80TM)    11
 Y좌표(GRS80TM)    11
 X좌표(WGS84)      11
 Y좌표(WGS84)      11
 dtype: int64,
 ID               0
 bus_stop_name    0
 latitude         0
 longitude        0
 dtype: int64,
 school_name    0
 address        0
 latitude       0
 longitude      0
 dtype: int64,
 hospital_name    0
 address          0
 latitude         0
 longitude        0
 dtype: int64)

In [5]:
# 여전히 존재하는 미싱밸류 확인 및 처리를 위한 전처리
# mv_school = school_df[school_df.isnull().any(axis=1)]
# mv_hospital = hospital_df[hospital_df.isnull().any(axis=1)]
# mv_school, mv_hospital
# mv_school.to_csv('forDP/mv_school.csv', index=False, encoding='utf-8')
# mv_hospital.to_csv('forDP/mv_hospital.csv', index=False, encoding='utf-8')

In [6]:
# 결측치 처리
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), \
    station_2019_df.isna().sum(), station_2020_df.isna().sum(), station_2021_df.isna().sum(), station_2022_df.isna().sum(), station_2023_df.isna().sum(), \
        school_df.isna().sum(), hospital_df.isna().sum()
park_df = park_df.dropna()
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), \
    station_2019_df.isna().sum(), station_2020_df.isna().sum(), station_2021_df.isna().sum(), station_2022_df.isna().sum(), station_2023_df.isna().sum(), \
        school_df.isna().sum(), hospital_df.isna().sum()
    

(NO                     0
 contract_date          0
 exclusive_area_m2      0
 floor                  0
 brand_label_encoded    0
 Q_1                    0
 Q_2                    0
 Q_3                    0
 Q_4                    0
 apt_age                0
 latitude               0
 longitude              0
 adjusted_price         0
 dtype: int64,
 공원명             0
 X좌표(GRS80TM)    0
 Y좌표(GRS80TM)    0
 X좌표(WGS84)      0
 Y좌표(WGS84)      0
 dtype: int64,
 ID               0
 bus_stop_name    0
 latitude         0
 longitude        0
 dtype: int64,
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 school_name    0
 address        0
 latitude  

In [7]:
# 컬럼 정리
apt_df = apt_df[['NO', 'contract_date', 'latitude', 'longitude']]

park_df.columns = ['park_name', 'x', 'y', 'longitude', 'latitude']
park_df = park_df[['park_name', 'latitude', 'longitude']]

bus_df.columns = ['bus_id', 'bus_stop_name', 'latitude', 'longitude']

station_2019_df['line_name'] = station_2019_df['line'] + '_' + station_2019_df['name']
station_2020_df['line_name'] = station_2020_df['line'] + '_' + station_2020_df['name']
station_2021_df['line_name'] = station_2021_df['line'] + '_' + station_2021_df['name']
station_2022_df['line_name'] = station_2022_df['line'] + '_' + station_2022_df['name']
station_2023_df['line_name'] = station_2023_df['line'] + '_' + station_2023_df['name']
station_2019_df = station_2019_df.drop_duplicates(subset='line_name')
station_2020_df = station_2020_df.drop_duplicates(subset='line_name')
station_2021_df = station_2021_df.drop_duplicates(subset='line_name')
station_2022_df = station_2022_df.drop_duplicates(subset='line_name')
station_2023_df = station_2023_df.drop_duplicates(subset='line_name')
station_2019_df.columns = ['station_name', 'line', 'latitude', 'longitude', 'line_name']
station_2020_df.columns = ['station_name', 'line', 'latitude', 'longitude', 'line_name']
station_2021_df.columns = ['station_name', 'line', 'latitude', 'longitude', 'line_name']
station_2022_df.columns = ['station_name', 'line', 'latitude', 'longitude', 'line_name']
station_2023_df.columns = ['station_name', 'line', 'latitude', 'longitude', 'line_name']
station_2019_df = station_2019_df[['line_name', 'latitude', 'longitude']]
station_2020_df = station_2020_df[['line_name', 'latitude', 'longitude']]
station_2021_df = station_2021_df[['line_name', 'latitude', 'longitude']]
station_2022_df = station_2022_df[['line_name', 'latitude', 'longitude']]
station_2023_df = station_2023_df[['line_name', 'latitude', 'longitude']]

school_df = school_df[['school_name', 'latitude', 'longitude']]

hospital_df = hospital_df[['hospital_name', 'latitude', 'longitude']]

In [8]:
apt_df.describe(), bus_df.describe(), park_df.describe(), school_df.describe(), hospital_df.describe(), \
station_2019_df.describe(), station_2020_df.describe(), station_2021_df.describe(), station_2022_df.describe(), station_2023_df.describe()

(            latitude      longitude
 count  251096.000000  251096.000000
 mean       37.555110     126.995693
 std         0.056496       0.090399
 min        37.434420     126.806400
 25%        37.506884     126.917908
 50%        37.549193     127.017588
 75%        37.599466     127.065709
 max        37.687503     127.179998,
              bus_id      latitude     longitude
 count  1.129000e+04  11290.000000  11290.000000
 mean   1.132294e+08     37.550071    126.985975
 std    6.993850e+06      0.054553      0.086257
 min    1.000000e+08     37.430537    126.721031
 25%    1.079001e+08     37.502696    126.916959
 50%    1.139001e+08     37.549471    126.993971
 75%    1.199001e+08     37.589371    127.051107
 max    1.670006e+08     37.690177    127.181760,
          latitude   longitude
 count  119.000000  119.000000
 mean    37.550757  126.984929
 std      0.057183    0.091444
 min     37.422435  126.806828
 25%     37.509223  126.912424
 50%     37.551714  126.984287
 75%   

In [9]:
apt_df['contract_date'] = pd.to_datetime(apt_df['contract_date'])

In [10]:
def calculate_nearest_facilities_with_scores(apt_df, ref_df, ref_name, radius_km_list=[0.5, 1, 1.5], 
                                             scores=[10, 5, 1], name_column='name'):
    """
    아파트 기준으로 가장 가까운 시설 이름 및 거리 계산 (중복 방지) + 점수 계산 추가.
    apt_df: 아파트 데이터프레임
    ref_df: 기준 데이터프레임 (공원, 버스정류장, 등)
    ref_name: 기준 이름 ('공원', '버스정류장', 등)
    radius_km_list: 계산할 반경 (km 단위)
    scores: 반경별 가중치 점수 (반경 순서에 맞게 입력)
    name_column: 시설 이름 컬럼 (기본값 'name')
    """
    # 결과 저장용 딕셔너리
    results = {f'{ref_name}_within_{r}km': [] for r in radius_km_list}
    results[f'closest_{ref_name}_dist_km'] = []
    results[f'closest_{ref_name}_name'] = []
    total_score_col = f'{ref_name}_score'  # 점수 컬럼 이름
    results[total_score_col] = []

    # KDTree 생성 (ref_df 기준 좌표를 라디안으로 변환)
    ref_tree = cKDTree(np.radians(ref_df[['latitude', 'longitude']].values))

    for _, apt_row in apt_df.iterrows():
        apt_coords = np.radians([apt_row['latitude'], apt_row['longitude']])  # 아파트 좌표 라디안 변환
        seen_indices = set()  # 중복 방지용 인덱스 저장
        total_score = 0  # 점수 초기화

        for radius_km, score in zip(radius_km_list, scores):
            radius_rad = radius_km / 6371.0  # 반경을 라디안 단위로 변환
            indices = ref_tree.query_ball_point(apt_coords, r=radius_rad)

            # 반경 내 고유 시설 개수 계산
            unique_indices = [idx for idx in indices if idx not in seen_indices]
            seen_indices.update(unique_indices)  # 이미 처리한 인덱스 추가

            count = len(unique_indices)
            results[f'{ref_name}_within_{radius_km}km'].append(count)

            # 점수 계산 (반경 내 개수 * 가중치)
            total_score += count * score

        # 가장 가까운 거리 및 인덱스 계산
        dist, idx = ref_tree.query(apt_coords, k=1)
        closest_distance = dist * 6371.0  # 라디안 -> km 변환
        closest_name = ref_df.iloc[idx][name_column]  # 인덱스를 통해 시설 이름 가져오기

        results[f'closest_{ref_name}_dist_km'].append(closest_distance)
        results[f'closest_{ref_name}_name'].append(closest_name)
        results[total_score_col].append(total_score)  # 최종 점수 저장

    # 결과를 apt_df에 추가
    for col_name, col_values in results.items():
        apt_df[col_name] = col_values

    return apt_df


In [11]:
# 버스정류장 (반경 500m, 1km, 1.5km -> 점수 10, 5, 1)
apt_df = calculate_nearest_facilities_with_scores(apt_df, bus_df, 'bus_stop', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='bus_stop_name')
# 공원 (반경 500m, 1km, 1.5km -> 점수 10, 5, 1)
apt_df = calculate_nearest_facilities_with_scores(apt_df, park_df, 'park', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='park_name')
# 병원
apt_df = calculate_nearest_facilities_with_scores(apt_df, hospital_df, 'hospital', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='hospital_name')
# 학교
apt_df = calculate_nearest_facilities_with_scores(apt_df, school_df, 'school', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='school_name')

In [12]:
apt_without_station = apt_df
apt_without_station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   latitude                  251096 non-null  float64       
 3   longitude                 251096 non-null  float64       
 4   bus_stop_within_0.5km     251096 non-null  int64         
 5   bus_stop_within_1km       251096 non-null  int64         
 6   bus_stop_within_1.5km     251096 non-null  int64         
 7   closest_bus_stop_dist_km  251096 non-null  float64       
 8   closest_bus_stop_name     251096 non-null  object        
 9   bus_stop_score            251096 non-null  int64         
 10  park_within_0.5km         251096 non-null  int64         
 11  park_within_1km           251096 non-null  int64         
 12  pa

In [14]:
apt_df['year'] = apt_df['contract_date'].dt.year
apt_df = apt_df[['NO', 'year', 'latitude', 'longitude']]

apt_2019 = apt_df[apt_without_station['year'] == 2019]
apt_2020 = apt_df[apt_without_station['year'] == 2020]
apt_2021 = apt_df[apt_without_station['year'] == 2021]
apt_2022 = apt_df[apt_without_station['year'] == 2022]
apt_2023 = apt_df[apt_without_station['year'] == 2023]

In [15]:
def calculate_apartment_station_distances_and_scores(apt_df, station_df, radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1]):
    """
    아파트를 기준으로 가장 가까운 지하철역 거리 및 점수 계산
    - apt_df: 아파트 데이터프레임 (latitude, longitude 필요)
    - station_df: 지하철 데이터프레임 (latitude, longitude, line_name 필요)
    - radius_km_list: 반경별 거리 범위 리스트 (km 단위)
    - scores: 거리 범위별 가중치 점수 리스트
    """
    # 결과 저장용 딕셔너리
    results = {
        f'station_within_{int(r * 1000)}m': [] for r in radius_km_list
    }
    results['closest_station_dist_km'] = []
    results['closest_station_name'] = []
    results['station_score'] = []  # 총 점수

    # KDTree 생성 (지하철 역 데이터)
    station_tree = cKDTree(np.radians(station_df[['latitude', 'longitude']].values))

    # 각 아파트의 좌표를 라디안으로 변환
    for _, apt_row in apt_df.iterrows():
        apt_coords = np.radians([apt_row['latitude'], apt_row['longitude']])  # 아파트 좌표
        seen_indices = set()  # 중복 방지용 인덱스 저장
        total_score = 0  # 점수 초기화

        # 반경별 거리 계산 및 점수 집계
        for radius_km, score in zip(radius_km_list, scores):
            radius_rad = radius_km / 6371.0  # 반경을 라디안으로 변환
            indices = station_tree.query_ball_point(apt_coords, r=radius_rad)

            # 중복되지 않은 고유 시설 개수 계산
            unique_indices = [idx for idx in indices if idx not in seen_indices]
            seen_indices.update(unique_indices)  # 처리한 인덱스 기록

            # 반경 내 고유 지하철역 개수 저장
            results[f'station_within_{int(radius_km * 1000)}m'].append(len(unique_indices))

            # 점수 추가
            total_score += len(unique_indices) * score

        # 가장 가까운 지하철 역 거리 및 이름
        dist, idx = station_tree.query(apt_coords, k=1)
        closest_distance_km = dist * 6371.0  # 라디안 -> km 변환
        closest_station_name = station_df.iloc[idx]['line_name']  # 지하철 역 이름

        results['closest_station_dist_km'].append(closest_distance_km)
        results['closest_station_name'].append(closest_station_name)
        results['station_score'].append(total_score)

    # 결과를 아파트 데이터프레임에 추가
    for col_name, col_values in results.items():
        apt_df[col_name] = col_values

    return apt_df


In [16]:
years = range(2019, 2024)
radius_km_list = [0.5, 1, 1.5]  # 반경 리스트 (km)
scores = [10, 5, 1]  # 반경별 가중치 점수
results = {}

for year in years:
    apt_df = globals()[f"apt_{year}"]  # 연도별 아파트 데이터
    station_df = globals()[f"station_{year}_df"]  # 연도별 지하철 데이터
    
    # 거리 계산 및 점수 집계
    results[year] = calculate_apartment_station_distances_and_scores(
        apt_df=apt_df,
        station_df=station_df,
        radius_km_list=radius_km_list,
        scores=scores
    )

# 연도별 결과 출력
for year in years:
    print(f"{year}년도 처리 결과:")
    print(results[year].head())


2019년도 처리 결과:
           NO  year   latitude   longitude  station_within_500m  \
0  2019_00001  2019  37.569628  127.041524                    1   
1  2019_00002  2019  37.589283  127.072500                    0   
2  2019_00003  2019  37.574609  127.060129                    0   
3  2019_00004  2019  37.616686  127.001842                    0   
4  2019_00005  2019  37.576338  127.074573                    0   

   station_within_1000m  station_within_1500m  closest_station_dist_km  \
0                     4                     7                 0.424042   
1                     1                     1                 0.744370   
2                     0                     2                 1.202505   
3                     1                     1                 0.878111   
4                     0                     1                 1.384230   

  closest_station_name  station_score  
0               5호선_마장             37  
1               중앙선_중랑              6  
2              5호선

In [19]:
# 5개 데이터프레임을 concat으로 병합
apt_station_df = pd.concat([apt_2019, apt_2020, apt_2021, apt_2022, apt_2023], ignore_index=True)
apt_station_df.info()
apt_station_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   NO                       251096 non-null  object 
 1   year                     251096 non-null  int32  
 2   latitude                 251096 non-null  float64
 3   longitude                251096 non-null  float64
 4   station_within_500m      251096 non-null  int64  
 5   station_within_1000m     251096 non-null  int64  
 6   station_within_1500m     251096 non-null  int64  
 7   closest_station_dist_km  251096 non-null  float64
 8   closest_station_name     251096 non-null  object 
 9   station_score            251096 non-null  int64  
dtypes: float64(3), int32(1), int64(4), object(2)
memory usage: 18.2+ MB


Index(['NO', 'year', 'latitude', 'longitude', 'station_within_500m',
       'station_within_1000m', 'station_within_1500m',
       'closest_station_dist_km', 'closest_station_name', 'station_score'],
      dtype='object')

In [20]:
apt_without_station.columns

Index(['NO', 'contract_date', 'latitude', 'longitude', 'bus_stop_within_0.5km',
       'bus_stop_within_1km', 'bus_stop_within_1.5km',
       'closest_bus_stop_dist_km', 'closest_bus_stop_name', 'bus_stop_score',
       'park_within_0.5km', 'park_within_1km', 'park_within_1.5km',
       'closest_park_dist_km', 'closest_park_name', 'park_score',
       'hospital_within_0.5km', 'hospital_within_1km', 'hospital_within_1.5km',
       'closest_hospital_dist_km', 'closest_hospital_name', 'hospital_score',
       'school_within_0.5km', 'school_within_1km', 'school_within_1.5km',
       'closest_school_dist_km', 'closest_school_name', 'school_score',
       'year'],
      dtype='object')

In [22]:
apt_all_distance = pd.merge(apt_without_station, 
                            apt_station_df[['NO', 'station_within_500m', 'station_within_1000m', 
                                            'station_within_1500m', 'closest_station_dist_km', 
                                            'closest_station_name', 'station_score']], 
                            how='inner', on='NO')
apt_all_distance.info()
apt_all_distance.head(), apt_all_distance.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   latitude                  251096 non-null  float64       
 3   longitude                 251096 non-null  float64       
 4   bus_stop_within_0.5km     251096 non-null  int64         
 5   bus_stop_within_1km       251096 non-null  int64         
 6   bus_stop_within_1.5km     251096 non-null  int64         
 7   closest_bus_stop_dist_km  251096 non-null  float64       
 8   closest_bus_stop_name     251096 non-null  object        
 9   bus_stop_score            251096 non-null  int64         
 10  park_within_0.5km         251096 non-null  int64         
 11  park_within_1km           251096 non-null  int64         
 12  pa

(           NO contract_date   latitude   longitude  bus_stop_within_0.5km  \
 0  2019_00001    2019-12-31  37.569628  127.041524                     21   
 1  2019_00002    2019-12-31  37.589283  127.072500                     13   
 2  2019_00003    2019-12-31  37.574609  127.060129                     15   
 3  2019_00004    2019-12-31  37.616686  127.001842                     11   
 4  2019_00005    2019-12-31  37.576338  127.074573                     18   
 
    bus_stop_within_1km  bus_stop_within_1.5km  closest_bus_stop_dist_km  \
 0                   40                     99                  0.177637   
 1                   49                     70                  0.139836   
 2                   29                     80                  0.214917   
 3                   38                     36                  0.147960   
 4                   47                     78                  0.208754   
 
   closest_bus_stop_name  bus_stop_score  ...  closest_school_dist_km  \

In [23]:
apt_all_distance['total_score'] = apt_all_distance['station_score'] + \
    apt_all_distance['bus_stop_score'] + apt_all_distance['park_score'] \
                    + apt_all_distance['hospital_score'] + apt_all_distance['school_score']

In [25]:
apt_all_distance.isna().sum(), \
apt_all_distance.columns

(NO                          0
 contract_date               0
 latitude                    0
 longitude                   0
 bus_stop_within_0.5km       0
 bus_stop_within_1km         0
 bus_stop_within_1.5km       0
 closest_bus_stop_dist_km    0
 closest_bus_stop_name       0
 bus_stop_score              0
 park_within_0.5km           0
 park_within_1km             0
 park_within_1.5km           0
 closest_park_dist_km        0
 closest_park_name           0
 park_score                  0
 hospital_within_0.5km       0
 hospital_within_1km         0
 hospital_within_1.5km       0
 closest_hospital_dist_km    0
 closest_hospital_name       0
 hospital_score              0
 school_within_0.5km         0
 school_within_1km           0
 school_within_1.5km         0
 closest_school_dist_km      0
 closest_school_name         0
 school_score                0
 year                        0
 station_within_500m         0
 station_within_1000m        0
 station_within_1500m        0
 closest

In [32]:
selected_apt_df = apt_all_distance[['NO', 'contract_date', 
                                    'closest_bus_stop_dist_km', 'bus_stop_score',
                                    'closest_park_dist_km', 'park_score',
                                    'closest_hospital_dist_km', 'hospital_score',
                                    'closest_school_dist_km', 'school_score', 
                                    'closest_station_dist_km', 'station_score',
                                    'total_score', 'latitude', 'longitude']]

not_selected_apt_df = apt_all_distance[['NO', 'contract_date', 'bus_stop_within_0.5km',
        'bus_stop_within_1km', 'bus_stop_within_1.5km', 'closest_bus_stop_name',
        'park_within_0.5km', 'park_within_1km', 'park_within_1.5km', 'closest_park_name', 
        'hospital_within_0.5km', 'hospital_within_1km', 'hospital_within_1.5km',
        'closest_hospital_name', 'school_within_0.5km', 'school_within_1km', 'school_within_1.5km',
        'closest_school_name', 'station_within_500m', 'station_within_1000m', 
        'station_within_1500m', 'closest_station_name', 'year']]

all_apt_distance = apt_all_distance

In [27]:
selected_apt_df.head()

,NO,contract_date,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,total_score,latitude,longitude
0,2019_00001,2019-12-31,0.177637,509,0.493332,15,0.088037,682,0.143573,45,0.424042,37,1288,37.569628,127.041524
1,2019_00002,2019-12-31,0.139836,445,1.288785,1,0.431829,514,0.336639,65,0.744370,6,1031,37.589283,127.072500
2,2019_00003,2019-12-31,0.214917,375,0.285478,16,0.000000,977,0.233091,71,1.202505,2,1441,37.574609,127.060129
3,2019_00004,2019-12-31,0.147960,336,0.880074,5,0.125651,119,0.536320,30,0.878111,6,496,37.616686,127.001842
4,2019_00005,2019-12-31,0.208754,493,1.279134,2,0.199009,785,0.200016,42,1.384230,1,1323,37.576338,127.074573


In [28]:
selected_apt_df.to_csv('Data_Preprocessing/apt_distance_with_score.csv', index = False, encoding='utf-8')

In [29]:
apt_df_withou_distance = pd.read_csv('Data_Preprocessing/apt_without_distance.csv')

In [31]:
apt_df_withou_distance.columns, selected_apt_df.columns

(Index(['NO', 'contract_date', 'exclusive_area_m2', 'floor',
        'brand_label_encoded', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 'apt_age',
        'latitude', 'longitude', 'adjusted_price'],
       dtype='object'),
 Index(['NO', 'contract_date', 'closest_bus_stop_dist_km', 'bus_stop_score',
        'closest_park_dist_km', 'park_score', 'closest_hospital_dist_km',
        'hospital_score', 'closest_school_dist_km', 'school_score',
        'closest_station_dist_km', 'station_score', 'total_score', 'latitude',
        'longitude'],
       dtype='object'))

In [53]:
all_apt = pd.merge(selected_apt_df, 
                   apt_df_withou_distance[['NO', 'exclusive_area_m2', 'floor',
                                           'brand_label_encoded', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 
                                           'apt_age', 'adjusted_price']], 
                   how = 'outer', on = 'NO')
all_apt

,NO,contract_date,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,...,longitude,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,adjusted_price
0,2019_00001,2019-12-31,0.177637,509,0.493332,15,0.088037,682,0.143573,45,...,127.041524,134.790,17,1,0,0,0,1,21,98701.507538
1,2019_00002,2019-12-31,0.139836,445,1.288785,1,0.431829,514,0.336639,65,...,127.072500,84.980,9,1,0,0,0,1,21,53612.864322
2,2019_00003,2019-12-31,0.214917,375,0.285478,16,0.000000,977,0.233091,71,...,127.060129,59.740,21,1,0,0,0,1,19,61127.638191
3,2019_00004,2019-12-31,0.147960,336,0.880074,5,0.125651,119,0.536320,30,...,127.001842,71.500,2,1,0,0,0,1,42,36115.778894
4,2019_00005,2019-12-31,0.208754,493,1.279134,2,0.199009,785,0.200016,42,...,127.074573,59.928,12,4,0,0,0,1,12,73465.326633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251091,2023_35638,2023-01-02,0.120570,681,0.989016,6,0.050423,1813,0.400552,53,...,126.921087,14.030,11,1,1,0,0,0,3,17000.000000
251092,2023_35639,2023-01-02,0.047301,549,1.144110,1,0.056842,1480,0.367395,40,...,126.926435,29.990,9,1,1,0,0,0,2,40000.000000
251093,2023_35640,2023-01-01,0.179996,295,0.780690,5,0.000000,356,0.179475,69,...,127.084798,99.790,6,1,1,0,0,0,30,175000.000000
251094,2023_35641,2023-01-01,0.179996,295,0.780690,5,0.000000,356,0.179475,69,...,127.084798,99.790,6,1,1,0,0,0,30,175000.000000


In [61]:
all_apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   closest_bus_stop_dist_km  251096 non-null  float64       
 3   bus_stop_score            251096 non-null  int64         
 4   closest_park_dist_km      251096 non-null  float64       
 5   park_score                251096 non-null  int64         
 6   closest_hospital_dist_km  251096 non-null  float64       
 7   hospital_score            251096 non-null  int64         
 8   closest_school_dist_km    251096 non-null  float64       
 9   school_score              251096 non-null  int64         
 10  closest_station_dist_km   251096 non-null  float64       
 11  station_score             251096 non-null  int64         
 12  to

In [54]:
all_apt.isna().sum(), all_apt.columns

(NO                          0
 contract_date               0
 closest_bus_stop_dist_km    0
 bus_stop_score              0
 closest_park_dist_km        0
 park_score                  0
 closest_hospital_dist_km    0
 hospital_score              0
 closest_school_dist_km      0
 school_score                0
 closest_station_dist_km     0
 station_score               0
 total_score                 0
 latitude                    0
 longitude                   0
 exclusive_area_m2           0
 floor                       0
 brand_label_encoded         0
 Q_1                         0
 Q_2                         0
 Q_3                         0
 Q_4                         0
 apt_age                     0
 adjusted_price              0
 dtype: int64,
 Index(['NO', 'contract_date', 'closest_bus_stop_dist_km', 'bus_stop_score',
        'closest_park_dist_km', 'park_score', 'closest_hospital_dist_km',
        'hospital_score', 'closest_school_dist_km', 'school_score',
        'closest_stati

In [55]:
all_apt.to_csv('Data_Preprocessing/apt_information_with_distance_and_not_distance.csv', index = False, encoding='utf-8')

In [56]:
politics_df = pd.read_csv('정치.csv')

In [57]:
politics_df.columns = ['date', 'K_progressive_president', 'K_conservative_president',
                       'K_progressive_majority', 'K_conservative_majority',
                       'A_progressive_president', 'A_conservative_president',
                       'A_progressive_senate', 'A_conservative_senate', 'A_progressive_house',
                       'A_conservative_house']
politics_df['date'] = pd.to_datetime(politics_df['date'])

In [58]:
merged_df = pd.merge(all_apt, politics_df, how='outer', left_on='contract_date', right_on='date')

In [59]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251097 entries, 0 to 251096
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   closest_bus_stop_dist_km  251096 non-null  float64       
 3   bus_stop_score            251096 non-null  float64       
 4   closest_park_dist_km      251096 non-null  float64       
 5   park_score                251096 non-null  float64       
 6   closest_hospital_dist_km  251096 non-null  float64       
 7   hospital_score            251096 non-null  float64       
 8   closest_school_dist_km    251096 non-null  float64       
 9   school_score              251096 non-null  float64       
 10  closest_station_dist_km   251096 non-null  float64       
 11  station_score             251096 non-null  float64       
 12  to

In [60]:
merged_df.isna().sum()

NO                          1
contract_date               1
closest_bus_stop_dist_km    1
bus_stop_score              1
closest_park_dist_km        1
park_score                  1
closest_hospital_dist_km    1
hospital_score              1
closest_school_dist_km      1
school_score                1
closest_station_dist_km     1
station_score               1
total_score                 1
latitude                    1
longitude                   1
exclusive_area_m2           1
floor                       1
brand_label_encoded         1
Q_1                         1
Q_2                         1
Q_3                         1
Q_4                         1
apt_age                     1
adjusted_price              1
date                        0
K_progressive_president     0
K_conservative_president    0
K_progressive_majority      0
K_conservative_majority     0
A_progressive_president     0
A_conservative_president    0
A_progressive_senate        0
A_conservative_senate       0
A_progress

In [62]:
pd.set_option('display.max_columns', None)  # 모든 컬럼 표시
pd.set_option('display.width', None)  # 화면 너비 제한 해제
print(merged_df[merged_df['NO'].isnull()])

         NO contract_date  closest_bus_stop_dist_km  bus_stop_score  \
212740  NaN           NaT                       NaN             NaN   

        closest_park_dist_km  park_score  closest_hospital_dist_km  \
212740                   NaN         NaN                       NaN   

        hospital_score  closest_school_dist_km  school_score  \
212740             NaN                     NaN           NaN   

        closest_station_dist_km  station_score  total_score  latitude  \
212740                      NaN            NaN          NaN       NaN   

        longitude  exclusive_area_m2  floor  brand_label_encoded  Q_1  Q_2  \
212740        NaN                NaN    NaN                  NaN  NaN  NaN   

        Q_3  Q_4  apt_age  adjusted_price       date  K_progressive_president  \
212740  NaN  NaN      NaN             NaN 2022-09-11                        0   

        K_conservative_president  K_progressive_majority  \
212740                         1                       1   


In [68]:
result = all_apt[(all_apt['contract_date'].dt.year == 2022) & 
                 (all_apt['contract_date'].dt.month == 9) & 
                 (all_apt['contract_date'].dt.day == 11)]
result

,NO,contract_date,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,total_score,latitude,longitude,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,adjusted_price


In [70]:
merged_df = merged_df.dropna(subset='NO')
merged_df.isna().sum()

NO                          0
contract_date               0
closest_bus_stop_dist_km    0
bus_stop_score              0
closest_park_dist_km        0
park_score                  0
closest_hospital_dist_km    0
hospital_score              0
closest_school_dist_km      0
school_score                0
closest_station_dist_km     0
station_score               0
total_score                 0
latitude                    0
longitude                   0
exclusive_area_m2           0
floor                       0
brand_label_encoded         0
Q_1                         0
Q_2                         0
Q_3                         0
Q_4                         0
apt_age                     0
adjusted_price              0
date                        0
K_progressive_president     0
K_conservative_president    0
K_progressive_majority      0
K_conservative_majority     0
A_progressive_president     0
A_conservative_president    0
A_progressive_senate        0
A_conservative_senate       0
A_progress

In [75]:
merged_df.corr().tail(12)

,NO,contract_date,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,total_score,latitude,longitude,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,adjusted_price,date,K_progressive_president,K_conservative_president,K_progressive_majority,K_conservative_majority,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house
adjusted_price,0.058904,0.086869,0.115237,-0.256993,0.067675,-0.070418,-0.090603,0.198739,-0.019440,-0.107961,-0.112917,0.163700,0.119374,-0.330289,0.133496,0.631126,0.188610,0.208638,-0.070166,0.009013,0.038492,0.015548,0.005115,1.000000,0.086869,-0.042815,0.042815,NaN,NaN,0.078209,-0.078209,0.078209,-0.078209,-0.050109,0.050109
date,0.936955,1.000000,-0.000331,0.004796,0.003436,-0.003317,-0.028212,0.018850,0.009162,-0.022548,-0.030240,0.034762,0.018278,-0.023594,-0.007932,-0.054503,0.031091,0.046292,-0.021712,0.032416,0.041572,-0.055419,-0.000882,0.086869,1.000000,-0.839071,0.839071,NaN,NaN,0.839210,-0.839210,0.839210,-0.839210,-0.796012,0.796012
K_progressive_president,-0.810710,-0.839071,-0.005133,0.001674,0.001816,0.005009,0.036234,-0.024839,0.005751,0.010892,0.035732,-0.041160,-0.022682,0.027858,-0.013738,0.037298,-0.040848,-0.056437,0.032921,-0.043368,-0.046634,0.061503,0.006111,-0.042815,-0.839071,1.000000,-1.000000,NaN,NaN,-0.613141,0.613141,-0.613141,0.613141,0.898579,-0.898579
K_conservative_president,0.810710,0.839071,0.005133,-0.001674,-0.001816,-0.005009,-0.036234,0.024839,-0.005751,-0.010892,-0.035732,0.041160,0.022682,-0.027858,0.013738,-0.037298,0.040848,0.056437,-0.032921,0.043368,0.046634,-0.061503,-0.006111,0.042815,0.839071,-1.000000,1.000000,NaN,NaN,0.613141,-0.613141,0.613141,-0.613141,-0.898579,0.898579
K_progressive_majority,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K_conservative_majority,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A_progressive_president,0.816158,0.839210,-0.001770,0.005753,0.007233,-0.009748,-0.027054,0.023346,0.019911,-0.030947,-0.026870,0.029903,0.022261,-0.027923,-0.012154,-0.063715,0.024970,0.023560,0.074240,0.085510,0.025357,-0.180170,-0.008803,0.078209,0.839210,-0.613141,0.613141,NaN,NaN,1.000000,-1.000000,1.000000,-1.000000,-0.550452,0.550452
A_conservative_president,-0.816158,-0.839210,0.001770,-0.005753,-0.007233,0.009748,0.027054,-0.023346,-0.019911,0.030947,0.026870,-0.029903,-0.022261,0.027923,0.012154,0.063715,-0.024970,-0.023560,-0.074240,-0.085510,-0.025357,0.180170,0.008803,-0.078209,-0.839210,0.613141,-0.613141,NaN,NaN,-1.000000,1.000000,-1.000000,1.000000,0.550452,-0.550452
A_progressive_senate,0.816158,0.839210,-0.001770,0.005753,0.007233,-0.009748,-0.027054,0.023346,0.019911,-0.030947,-0.026870,0.029903,0.022261,-0.027923,-0.012154,-0.063715,0.024970,0.023560,0.074240,0.085510,0.025357,-0.180170,-0.008803,0.078209,0.839210,-0.613141,0.613141,NaN,NaN,1.000000,-1.000000,1.000000,-1.000000,-0.550452,0.550452
A_conservative_senate,-0.816158,-0.839210,0.001770,-0.005753,-0.007233,0.009748,0.027054,-0.023346,-0.019911,0.030947,0.026870,-0.029903,-0.022261,0.027923,0.012154,0.063715,-0.024970,-0.023560,-0.074240,-0.085510,-0.025357,0.180170,0.008803,-0.078209,-0.839210,0.613141,-0.613141,NaN,NaN,-1.000000,1.000000,-1.000000,1.000000,0.550452,-0.550452


In [126]:
merged_df.columns

Index(['NO', 'contract_date', 'closest_bus_stop_dist_km', 'bus_stop_score',
       'closest_park_dist_km', 'park_score', 'closest_hospital_dist_km',
       'hospital_score', 'closest_school_dist_km', 'school_score',
       'closest_station_dist_km', 'station_score', 'total_score', 'latitude',
       'longitude', 'exclusive_area_m2', 'floor', 'brand_label_encoded', 'Q_1',
       'Q_2', 'Q_3', 'Q_4', 'apt_age', 'adjusted_price', 'date',
       'K_progressive_president', 'K_conservative_president',
       'A_progressive_president', 'A_conservative_president',
       'A_progressive_senate', 'A_conservative_senate', 'A_progressive_house',
       'A_conservative_house', 'pyeong_price'],
      dtype='object')

In [77]:
merged_df = merged_df.drop(columns=['K_progressive_majority','K_conservative_majority'])

In [86]:
merged_df.columns

Index(['NO', 'contract_date', 'closest_bus_stop_dist_km', 'bus_stop_score',
       'closest_park_dist_km', 'park_score', 'closest_hospital_dist_km',
       'hospital_score', 'closest_school_dist_km', 'school_score',
       'closest_station_dist_km', 'station_score', 'total_score', 'latitude',
       'longitude', 'exclusive_area_m2', 'floor', 'brand_label_encoded', 'Q_1',
       'Q_2', 'Q_3', 'Q_4', 'apt_age', 'adjusted_price', 'date',
       'K_progressive_president', 'K_conservative_president',
       'A_progressive_president', 'A_conservative_president',
       'A_progressive_senate', 'A_conservative_senate', 'A_progressive_house',
       'A_conservative_house'],
      dtype='object')

In [111]:
# 'adjusted_price'를 'exclusive_area_m2'로 나누고 3.3058을 곱해 평으로 변환
merged_df['pyeong_price'] = merged_df['adjusted_price'] / (merged_df['exclusive_area_m2'] / 3.3058)

In [128]:
new_col = ['closest_bus_stop_dist_km', 'bus_stop_score',
       'closest_park_dist_km', 'park_score', 'closest_hospital_dist_km',
       'hospital_score', 'closest_school_dist_km', 'school_score',
       'closest_station_dist_km', 'station_score', 'latitude',
       'longitude', 'exclusive_area_m2', 'floor', 'brand_label_encoded', 'Q_1',
       'Q_2', 'Q_3', 'Q_4', 'apt_age', 
       'K_progressive_president', 'A_progressive_president', 'A_progressive_senate', 'A_progressive_house',
       'pyeong_price', 'adjusted_price']
corr = merged_df[new_col].corr()

model_t = merged_df[new_col]

In [129]:
# 상관관계 행렬 계산
corr_matrix = corr

# 상관관계 행렬을 오름차순으로 정렬 (평평하게 만들어서)
corr_sorted = corr_matrix.unstack()

# NaN 값 제거 및 정렬
corr_sorted = corr_sorted[corr_sorted != 1.0]  # 자기 자신과의 상관관계(1.0) 제외
corr_sorted = corr_sorted.sort_values()

print(corr_sorted.to_string())

park_score                closest_park_dist_km       -0.762188
closest_park_dist_km      park_score                 -0.762188
station_score             closest_station_dist_km    -0.630962
closest_station_dist_km   station_score              -0.630962
K_progressive_president   A_progressive_president    -0.613141
A_progressive_senate      K_progressive_president    -0.613141
K_progressive_president   A_progressive_senate       -0.613141
A_progressive_president   K_progressive_president    -0.613141
A_progressive_senate      A_progressive_house        -0.550452
A_progressive_house       A_progressive_president    -0.550452
                          A_progressive_senate       -0.550452
A_progressive_president   A_progressive_house        -0.550452
brand_label_encoded       apt_age                    -0.429878
apt_age                   brand_label_encoded        -0.429878
school_score              closest_school_dist_km     -0.425955
closest_school_dist_km    school_score               -0

In [130]:
model_t

,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,latitude,longitude,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,K_progressive_president,A_progressive_president,A_progressive_senate,A_progressive_house,pyeong_price,adjusted_price
0,0.086715,418.0,0.782697,5.0,0.065513,518.0,0.295451,59.0,0.811709,7.0,37.501962,126.865676,56.9700,6.0,1.0,1.0,0.0,0.0,0.0,23.0,1,0,0,0,1724.715013,29722.613065
1,0.167992,318.0,1.153430,3.0,0.000000,492.0,0.285764,49.0,0.154614,21.0,37.571027,126.806400,51.0300,9.0,1.0,1.0,0.0,0.0,0.0,25.0,1,0,0,0,2942.707825,45425.125628
2,0.069199,658.0,0.931005,5.0,0.240563,833.0,0.605951,47.0,0.513198,10.0,37.584667,126.914192,48.4800,6.0,1.0,1.0,0.0,0.0,0.0,1.0,1,0,0,0,3212.213244,47107.537688
3,0.170212,610.0,2.113483,0.0,0.055632,685.0,0.419592,59.0,0.430952,29.0,37.617218,127.062510,114.9900,2.0,1.0,1.0,0.0,0.0,0.0,19.0,1,0,0,0,2102.350928,73128.844221
4,0.134097,595.0,1.288343,1.0,0.206349,336.0,0.211266,50.0,0.346779,16.0,37.615142,127.007702,84.9700,4.0,4.0,1.0,0.0,0.0,0.0,11.0,1,0,0,0,2269.109294,58323.618090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251092,0.251083,552.0,0.865292,11.0,0.306460,658.0,0.346009,46.0,0.466759,11.0,37.497538,126.848289,59.5800,10.0,1.0,0.0,0.0,0.0,1.0,24.0,0,1,1,0,2580.055388,46500.000000
251093,0.124402,689.0,1.170883,1.0,0.000000,821.0,0.294393,78.0,0.183458,27.0,37.500420,126.911545,84.9700,14.0,4.0,0.0,0.0,0.0,1.0,6.0,0,1,1,0,5368.958456,138000.000000
251094,0.025612,785.0,0.991470,5.0,0.000000,915.0,0.568166,41.0,0.269433,19.0,37.504982,126.946788,84.9704,15.0,4.0,0.0,0.0,0.0,1.0,5.0,0,1,1,0,5738.533654,147500.000000
251095,0.081886,270.0,2.797163,0.0,0.121564,842.0,0.273683,43.0,0.563800,27.0,37.511820,127.006237,84.5400,2.0,1.0,0.0,0.0,0.0,1.0,24.0,0,1,1,0,7116.815709,182000.000000


In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 모델링
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score


In [132]:
from sklearn.model_selection import train_test_split

X = model_t.drop(columns = ['adjusted_price', 'pyeong_price'])
y = model_t['adjusted_price']

In [133]:
sample_df = model_t.sample(n=15000, random_state=42)
sample_df

X = sample_df.drop(columns=['adjusted_price', 'pyeong_price'])
y = sample_df['adjusted_price']

X_train,X_test, y_train,y_test = train_test_split(X,y, random_state= 42, test_size=0.25)

In [134]:
# 랜덤 포레스트 모델로 학습
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"Random Forest - MSE: {mse_rf:.2f}, R2 Score: {r2_rf:.2f}")
print(f"Random Forest RMSE: {rmse_rf:.2f}")

Random Forest - MSE: 506243962.26, R2 Score: 0.90
Random Forest RMSE: 22499.87


In [135]:
# MinMaxScaler로 스케일링
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 딕셔너리 생성
models = {
    'KNN': KNeighborsRegressor(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVM': SVR(),
}

# XGBoost 모델 추가 (xgboost 설치 필요)
try:
    models['XGBoost'] = xgb.XGBRegressor(random_state=42)
except:
    print("XGBoost 모델을 추가하지 못했습니다. xgboost 라이브러리를 설치해주세요.")

# 결과 저장할 딕셔너리
results = {}

# 모델별 학습 및 평가
for name, model in models.items():
    # 모델 학습
    model.fit(X_train_scaled, y_train)
    
    # 예측
    y_pred = model.predict(X_test_scaled)
    
    # 평가 지표 계산
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # 결과 저장
    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'R2 Score': r2
    }

# 결과 출력
for name, metrics in results.items():
    print(f"{name} 모델:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.2f}")
    print()

KNN 모델:
  MSE: 1624252523.65
  RMSE: 40302.02
  R2 Score: 0.66

Ridge 모델:
  MSE: 1959309754.20
  RMSE: 44264.09
  R2 Score: 0.59

Lasso 모델:
  MSE: 1960252126.12
  RMSE: 44274.73
  R2 Score: 0.59

DecisionTree 모델:
  MSE: 942029461.88
  RMSE: 30692.50
  R2 Score: 0.81

RandomForest 모델:
  MSE: 488778757.26
  RMSE: 22108.34
  R2 Score: 0.90

GradientBoosting 모델:
  MSE: 765883118.71
  RMSE: 27674.59
  R2 Score: 0.84

SVM 모델:
  MSE: 5132557266.38
  RMSE: 71641.87
  R2 Score: -0.06

XGBoost 모델:
  MSE: 488821315.41
  RMSE: 22109.30
  R2 Score: 0.90



In [136]:
# feature 가 얼마나 설명력 있는지 확인 제일 높은거 면적 0.4
for i, c in zip(rf_model.feature_importances_, X_train.columns):
    print("{0} : {1}".format(c, i))

closest_bus_stop_dist_km : 0.009745915288853735
bus_stop_score : 0.045021178574762975
closest_park_dist_km : 0.016358099862646105
park_score : 0.0025768838084217916
closest_hospital_dist_km : 0.008639073225070923
hospital_score : 0.025381169531681437
closest_school_dist_km : 0.010081664516139559
school_score : 0.017979963658997605
closest_station_dist_km : 0.015867539120573742
station_score : 0.015426152529391836
latitude : 0.17886878390647554
longitude : 0.12854868910249825
exclusive_area_m2 : 0.4296388173786866
floor : 0.01179337276540027
brand_label_encoded : 0.012824668882464686
Q_1 : 0.001004833191163548
Q_2 : 0.0016505461305794857
Q_3 : 0.0014300814353303867
Q_4 : 0.0016874260085584082
apt_age : 0.05016415316080854
K_progressive_president : 0.0015327549773380498
A_progressive_president : 0.006891064449681278
A_progressive_senate : 0.006033754762590261
A_progressive_house : 0.0008534137318848711


In [137]:
from sklearn.model_selection import train_test_split

sample_df = model_t.sample(n=15000, random_state=42)
sample_df

X = sample_df.drop(columns=['adjusted_price', 'pyeong_price'])
y = sample_df['pyeong_price']

X_train,X_test, y_train,y_test = train_test_split(X,y, random_state= 42, test_size=0.25)

In [142]:
merged_df['pyeong_price'].describe()

count    251096.000000
mean       4230.544008
std        2133.190310
min         526.948275
25%        2779.866085
50%        3703.430353
75%        5078.947016
max       29129.459395
Name: pyeong_price, dtype: float64

In [138]:
# 랜덤 포레스트 모델로 학습
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"Random Forest - MSE: {mse_rf:.2f}, R2 Score: {r2_rf:.2f}")
print(f"Random Forest RMSE: {rmse_rf:.2f}")

Random Forest - MSE: 551746.23, R2 Score: 0.88
Random Forest RMSE: 742.80


In [139]:
# MinMaxScaler로 스케일링
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 딕셔너리 생성
models = {
    'KNN': KNeighborsRegressor(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVM': SVR(),
}

# XGBoost 모델 추가 (xgboost 설치 필요)
try:
    models['XGBoost'] = xgb.XGBRegressor(random_state=42)
except:
    print("XGBoost 모델을 추가하지 못했습니다. xgboost 라이브러리를 설치해주세요.")

# 결과 저장할 딕셔너리
results = {}

# 모델별 학습 및 평가
for name, model in models.items():
    # 모델 학습
    model.fit(X_train_scaled, y_train)
    
    # 예측
    y_pred = model.predict(X_test_scaled)
    
    # 평가 지표 계산
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # 결과 저장
    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'R2 Score': r2
    }

# 결과 출력
for name, metrics in results.items():
    print(f"{name} 모델:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.2f}")
    print()

KNN 모델:
  MSE: 1801978.94
  RMSE: 1342.38
  R2 Score: 0.62

Ridge 모델:
  MSE: 2951954.90
  RMSE: 1718.13
  R2 Score: 0.37

Lasso 모델:
  MSE: 2957946.66
  RMSE: 1719.87
  R2 Score: 0.37

DecisionTree 모델:
  MSE: 1043907.94
  RMSE: 1021.72
  R2 Score: 0.78

RandomForest 모델:
  MSE: 543054.80
  RMSE: 736.92
  R2 Score: 0.88

GradientBoosting 모델:
  MSE: 1196612.63
  RMSE: 1093.90
  R2 Score: 0.74

SVM 모델:
  MSE: 4804897.68
  RMSE: 2192.01
  R2 Score: -0.03

XGBoost 모델:
  MSE: 522852.01
  RMSE: 723.09
  R2 Score: 0.89



In [143]:
# 과대적합 판별을 위한 함수
def check_overfitting(model, X_train_scaled, X_test_scaled, y_train, y_test):
    # 훈련 데이터에서의 예측
    y_train_pred = model.predict(X_train_scaled)
    
    # 테스트 데이터에서의 예측
    y_test_pred = model.predict(X_test_scaled)
    
    # 훈련 세트 RMSE
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    # 테스트 세트 RMSE
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # RMSE 차이
    rmse_diff = abs(train_rmse - test_rmse)
    
    print(f"훈련 세트 RMSE: {train_rmse:.2f}")
    print(f"테스트 세트 RMSE: {test_rmse:.2f}")
    print(f"RMSE 차이: {rmse_diff:.2f}")
    
    # 과대적합 판별 기준 (경험적 기준)
    if train_rmse < test_rmse and rmse_diff > train_rmse * 0.3:
        print("⚠️ 과대적합 가능성이 높음")
    elif train_rmse > test_rmse:
        print("⚠️ 과소적합 가능성이 높음")
    else:
        print("✅ 적절한 모델 성능")

# 각 모델에 대해 과대적합 검사
for name, model in models.items():
    print(f"\n{name} 모델 과대적합 분석:")
    check_overfitting(model, X_train_scaled, X_test_scaled, y_train, y_test)


KNN 모델 과대적합 분석:
훈련 세트 RMSE: 1063.55
테스트 세트 RMSE: 1342.38
RMSE 차이: 278.82
✅ 적절한 모델 성능

Ridge 모델 과대적합 분석:
훈련 세트 RMSE: 1698.95
테스트 세트 RMSE: 1718.13
RMSE 차이: 19.17
✅ 적절한 모델 성능

Lasso 모델 과대적합 분석:
훈련 세트 RMSE: 1699.29
테스트 세트 RMSE: 1719.87
RMSE 차이: 20.58
✅ 적절한 모델 성능

DecisionTree 모델 과대적합 분석:
훈련 세트 RMSE: 20.02
테스트 세트 RMSE: 1021.72
RMSE 차이: 1001.70
⚠️ 과대적합 가능성이 높음

RandomForest 모델 과대적합 분석:
훈련 세트 RMSE: 270.88
테스트 세트 RMSE: 736.92
RMSE 차이: 466.04
⚠️ 과대적합 가능성이 높음

GradientBoosting 모델 과대적합 분석:
훈련 세트 RMSE: 1015.24
테스트 세트 RMSE: 1093.90
RMSE 차이: 78.66
✅ 적절한 모델 성능

SVM 모델 과대적합 분석:
훈련 세트 RMSE: 2161.97
테스트 세트 RMSE: 2192.01
RMSE 차이: 30.04
✅ 적절한 모델 성능

XGBoost 모델 과대적합 분석:
훈련 세트 RMSE: 399.05
테스트 세트 RMSE: 723.09
RMSE 차이: 324.04
⚠️ 과대적합 가능성이 높음


In [144]:
from sklearn.model_selection import cross_val_score

# 각 모델의 교차 검증 점수
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                                cv=5, scoring='neg_mean_squared_error')
    cv_rmse_scores = np.sqrt(-cv_scores)
    print(f"{name} 모델 교차검증 RMSE:")
    print(f"  평균: {cv_rmse_scores.mean():.2f}")
    print(f"  표준편차: {cv_rmse_scores.std():.2f}\n")

KNN 모델 교차검증 RMSE:
  평균: 1391.38
  표준편차: 67.92

Ridge 모델 교차검증 RMSE:
  평균: 1699.66
  표준편차: 92.35

Lasso 모델 교차검증 RMSE:
  평균: 1700.01
  표준편차: 93.02

DecisionTree 모델 교차검증 RMSE:
  평균: 1066.48
  표준편차: 25.59

RandomForest 모델 교차검증 RMSE:
  평균: 758.61
  표준편차: 18.25

GradientBoosting 모델 교차검증 RMSE:
  평균: 1067.09
  표준편차: 42.55

SVM 모델 교차검증 RMSE:
  평균: 2168.07
  표준편차: 105.04

XGBoost 모델 교차검증 RMSE:
  평균: 748.77
  표준편차: 21.56



In [159]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
 # 7 = 0.74,0.72 10 = 0.88, 0.82 13 = 0.94, 0.86
rf = RandomForestRegressor(n_estimators=100,max_depth=13,random_state=42)
rf.fit(X_train,y_train)

rf.score(X_train,y_train), rf.score(X_test,y_test)

y_pred_rf = rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print(f"Random Forest - MSE: {mse_rf:.2f}, R2 Score: {r2_rf:.2f}")
print(mae_rf, mae_rf / np.mean(y_test))

Random Forest - MSE: 617663.10, R2 Score: 0.87
559.55510675183 0.130647602431347
